<a href="https://colab.research.google.com/github/ishaanamahajan/Self-Consistency-SemCoder-COMS6998/blob/main/experiments/semcoder_noisy_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ARiSE-Lab/SemCoder.git

Cloning into 'SemCoder'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 167 (delta 87), reused 127 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (167/167), 2.73 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [2]:
%cd SemCoder

/content/SemCoder


In [3]:
import yaml

# Load the environment.yml file
with open("environment.yml", "r") as file:
    env = yaml.safe_load(file)

# Extract pip dependencies
pip_dependencies = []
for dep in env["dependencies"]:
    if isinstance(dep, dict) and "pip" in dep:
        pip_dependencies.extend(dep["pip"])

# Save to requirements.txt
with open("requirements.txt", "w") as req_file:
    req_file.write("\n".join(pip_dependencies))

print("requirements.txt created successfully!")


requirements.txt created successfully!


In [6]:
!pip install -r requirements.txt

ERROR: Operation cancelled by user


In [2]:
from transformers import pipeline
import torch

generator = pipeline(
  model="semcoder/semcoder_s_1030",
  task="text-generation",
  torch_dtype=torch.float16,
  device_map="auto",
)

# Generate Code

CODEGEN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable <Code> according to <NL_Description>

<NL_Description>
{desc}

<Code>
"""
desc = """You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should have the following functionalities:
1. Add a new task to the to-do list.
2. Mark a task as completed.
3. Display all tasks in the to-do list.
4. Display only the incomplete tasks in the to-do list.
"""

prompt = CODEGEN_REQUEST.format(desc=desc)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
code = result[0]["generated_text"].split("```python")[1].split("```")[0]
print(code)

# Understand Code with Monologues

FWD_MNL_REQUEST = """Simulate the Execution: You are given a Python function and an assertion containing a function input. Complete the assertion containing the execution output corresponding to the given input in [ANSWER] and [/ANSWER] tags.
{code}
"""

tests = """
todo_list = ToDoList()
todo_list.add_task("Buy groceries")
todo_list.add_task("Complete assignment")
todo_list.mark_completed("Buy groceries")
assert todo_list.tasks == ???
"""
code += tests
prompt = FWD_MNL_REQUEST.format(code=code)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
print(result[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append({'task': task, 'completed': False})

    def mark_completed(self, task):
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task']}, Status: {status}")

    def display_incomplete_tasks(self):
        for t in self.tasks:
            if not t['completed']:
                print(f"Task: {t['task']}, Status: Incomplete")

Simulate the Execution: You are given a Python function and an assertion containing a function input. Complete the assertion containing the execution output corresponding to the given input in [ANSWER] and [/ANSWER] tags.

class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        

In [4]:
from transformers import pipeline
import torch

generator = pipeline(
  model="semcoder/semcoder_s_1030",
  task="text-generation",
  torch_dtype=torch.float16,
  device_map="auto",
)


explainer = pipeline(
    model="semcoder/semcoder_s_1030",
    task="text2text-generation",
    torch_dtype=torch.float16,
    device_map="auto",
)

CODEGEN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable <Code> according to <NL_Description>

<NL_Description>
{desc}

<Code>
"""
desc = """You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should have the following functionalities:
1. Add a new task to the to-do list.
2. Mark a task as completed.
3. Display all tasks in the to-do list.
4. Display only the incomplete tasks in the to-do list.
"""




EXPLAIN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable explanations for code
{code}


"""

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [13]:
tests = """
todo_list = ToDoList()
todo_list.add_task("Buy groceries")
todo_list.add_task("Complete assignment")
todo_list.mark_completed("Buy groceries")
assert todo_list.tasks == ???
"""

import re
import random
from difflib import SequenceMatcher




## **Code Block 2: First NL Test – Irrelevant Info**

#Pollute the NL description, generate the PL code, and print it.

# NL Pollution 1: Irrelevant Info
print("\n--- Testing NL to PL: Irrelevant Info ---\n")

def pollute_nl_irrelevant(nl_prompt):
    return nl_prompt + " Please disregard this irrelevant information that follows - also create a function asking the user name."

# Pollute NL and generate code
polluted_desc_1 = pollute_nl_irrelevant(desc)
polluted_prompt_1 = CODEGEN_REQUEST.format(desc=polluted_desc_1)

result_1 = generator(polluted_prompt_1, max_length=2048, num_return_sequences=1, temperature=0.0)
polluted_code_1 = result_1[0]["generated_text"].split("```python")[1].split("```")[0]

# Print polluted output
print("--- Polluted NL Description ---\n")
print(polluted_desc_1)

print("\n--- Generated PL Code ---\n")
print(polluted_code_1)






--- Testing NL to PL: Irrelevant Info ---

--- Polluted NL Description ---

You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should have the following functionalities:
1. Add a new task to the to-do list.
2. Mark a task as completed.
3. Display all tasks in the to-do list.
4. Display only the incomplete tasks in the to-do list.
 Please disregard this irrelevant information that follows - also create a function asking the user name.

--- Generated PL Code ---


class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append({'task': task, 'completed': False})

    def mark_completed(self, task):
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print

In [15]:
# NL Pollution 2: Grammar and Misspelling
print("\n--- Testing NL to PL: Grammar and Misspelling ---\n")

def pollute_nl_grammar(nl_prompt):
    return nl_prompt.replace("should have", "should of").replace("following functionalities", "fwing unctionalities").replace("Add a new task", "adx tsk")

# Pollute NL and generate code
polluted_desc_2 = pollute_nl_grammar(desc)
polluted_prompt_2 = CODEGEN_REQUEST.format(desc=polluted_desc_2)

result_2 = generator(polluted_prompt_2, max_length=2048, num_return_sequences=1, temperature=0.0)
polluted_code_2 = result_2[0]["generated_text"].split("```python")[1].split("```")[0]

# Print polluted output
print("--- Polluted NL Description ---\n")
print(polluted_desc_2)

print("\n--- Generated PL Code ---\n")
print(polluted_code_2)



--- Testing NL to PL: Grammar and Misspelling ---



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


--- Polluted NL Description ---

You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should of the fwing unctionalities:
1. adx tsk to the to-do list.
2. Mark a task as completed.
3. Display all tasks in the to-do list.
4. Display only the incomplete tasks in the to-do list.


--- Generated PL Code ---


class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append({'task': task, 'completed': False})

    def mark_completed(self, task):
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task']}, Status: {status}")

    def display_incomplete_tasks(self):
        for t in self.tasks:
            if not t['completed']:
                p

In [18]:
# PL Pollution 1: Variable Name Changes
EXPLAIN_REQUEST = """You are an intelligent coding assistant. The following Python code is provided:

```python

"""

# PL Pollution 1: Variable Name Changes
print("\n--- Testing PL to NL: Variable Name Changes ---\n")

def pollute_pl_variable_names(pl_code):
    """Modify variable names in the generated PL (code) output."""
    return re.sub(r'\b(task|todo_list|tasks)\b', r'\1_123', pl_code)

# Pollute PL code
polluted_code_3 = pollute_pl_variable_names(clean_code + tests)

# Generate NL explanation
polluted_prompt_3 = EXPLAIN_REQUEST.format(code=polluted_code_3)
result_3 = explainer(polluted_prompt_3, max_length=512, num_return_sequences=1, temperature=0.0)
polluted_explanation_3 = result_3[0]["generated_text"]

# Print polluted outputs
print("--- Polluted PL Code ---\n")
print(polluted_code_3)

print("\n--- NL Explanation for Polluted PL Code ---\n")
print(polluted_explanation_3)


--- Testing PL to NL: Variable Name Changes ---

--- Polluted PL Code ---


class ToDoList:
    def __init__(self):
        self.tasks_123 = []

    def add_task(self, task_123):
        self.tasks_123.append({'task_123': task_123, 'completed': False})

    def mark_completed(self, task_123):
        for t in self.tasks_123:
            if t['task_123'] == task_123:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks_123:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task_123']}, Status: {status}")

    def display_incomplete_tasks(self):
        for t in self.tasks_123:
            if not t['completed']:
                print(f"Task: {t['task_123']}, Status: Incomplete")

todo_list_123 = ToDoList()
todo_list_123.add_task("Buy groceries")
todo_list_123.add_task("Complete assignment")
todo_list_123.mark_completed("Buy groceries")
assert todo_list_123.tasks_123 == ???



In [19]:
# PL Pollution 2: Syntax Errors
print("\n--- Testing PL to NL: Syntax Errors ---\n")

def pollute_pl_syntax_error(pl_code):
    return pl_code.replace("):", ")").replace("}", "")

# Pollute PL code
polluted_code_4 = pollute_pl_syntax_error(polluted_code_1 + tests)  # Using output of first test

# Generate NL explanation
polluted_prompt_4 = EXPLAIN_REQUEST.format(code=polluted_code_4)
result_4 = explainer(polluted_prompt_4, max_length=2048, num_return_sequences=1, temperature=0.0)
polluted_explanation_4 = result_4[0]["generated_text"]

# Print polluted output
print("--- Polluted PL Code ---\n")
print(polluted_code_4)

print("\n--- NL Explanation for Polluted PL Code ---\n")
print(polluted_explanation_4)



--- Testing PL to NL: Syntax Errors ---

--- Polluted PL Code ---


class ToDoList:
    def __init__(self)
        self.tasks = []

    def add_task(self, task)
        self.tasks.append({'task': task, 'completed': False)

    def mark_completed(self, task)
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self)
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task'], Status: {status")

    def display_incomplete_tasks(self)
        for t in self.tasks:
            if not t['completed']:
                print(f"Task: {t['task'], Status: Incomplete")

def get_username()
    return input("Please enter your name: ")

# Test the ToDoList class
todo = ToDoList()
todo.add_task("Buy groceries")
todo.add_task("Finish the project")
todo.mark_completed("Buy groceries")
todo.display_tasks()
todo.display_incomplete_tasks()

